Defining ML Model 

In [6]:
%pip install -r ..\requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [63]:

#initialise neural network
import torch
import torch.nn as nn

class model(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.mlp = nn.Sequential(
            #first layer
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.BatchNorm1d(64),
            nn.Dropout(0.5),
            #hidden layer
            nn.Linear(64, 8),
            nn.ReLU(),
            nn.BatchNorm1d(8),
            nn.Dropout(0.5),
            #output scalar score
            nn.Linear(8, 1)
        )

    def forward(self, x):
        return self.mlp(x)
    
embed_to_score = model(input_dim=512)
criterion = nn.MSELoss()
optimizer = torch.optim.AdamW(embed_to_score.parameters(), lr=1e-4, weight_decay=1e-4)

Data preprocessing

In [50]:
#entering data
import csv
import ast
import numpy as np
embeddings = []
scores = []

with open(r'C:\Users\rwwj8\OneDrive\Documents\scores_embeds.csv', 'r', encoding = 'utf-8') as file:
    reader = csv.DictReader(file)
    data = [row for row in reader]
    for row in data:
        rowlist = ast.literal_eval(row['embeds'])
        #print(len(rowlist))
        embeddings.append(rowlist)
        score = float(row['score'])
        scores.append(score)
embeddingarr = np.array(embeddings)
scorearr = np.array(scores)
print(scorearr.shape)

(469,)


In [51]:
#gpu code
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
embed_to_score.to(device)

model(
  (mlp): Sequential(
    (0): Linear(in_features=512, out_features=64, bias=True)
    (1): ReLU()
    (2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Dropout(p=0.5, inplace=False)
    (4): Linear(in_features=64, out_features=8, bias=True)
    (5): ReLU()
    (6): BatchNorm1d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): Dropout(p=0.5, inplace=False)
    (8): Linear(in_features=8, out_features=1, bias=True)
  )
)

In [32]:
#data preprocessing
from sklearn.model_selection import train_test_split
emb_train, score_train, emb_val, score_val = train_test_split(embeddingarr, scorearr, test_size=0.2, random_state=42)
#convert to tensors
emb_train = torch.FloatTensor(emb_train).to(device)
emb_val = torch.FloatTensor(emb_val).to(device)  # Add dim for MSE loss
score_train = torch.FloatTensor(score_train).to(device)
score_val = torch.FloatTensor(score_val).to(device)
print(emb_train.shape, emb_val.shape, score_train.shape, score_val.shape)
#data loader
from torch.utils.data import DataLoader, TensorDataset
train_dataset = TensorDataset(emb_train, emb_val)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataset = TensorDataset(score_train, score_val)
val_loader = DataLoader(val_dataset, batch_size=32)

torch.Size([375, 512]) torch.Size([375]) torch.Size([94, 512]) torch.Size([94])


Model training

In [52]:
#training loop
import torch.optim as optim

def train(model, dataloader, criterion, optimizer):
    model.train()
    total_loss = 0
    for X_batch, y_batch in tqdm(dataloader):
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(dataloader)

def validate(model, dataloader, criterion):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for X_batch, y_batch in dataloader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            total_loss += loss.item()
    return total_loss / len(dataloader)

In [65]:
from tqdm import tqdm
tqdm._instances.clear()

best_val_loss = float('inf')
patience = 50  # Stop after 5 epochs without improvement
epochs_no_improve = 0

for epoch in range(1000):  # Max epochs
    train_loss = train(embed_to_score, train_loader, criterion, optimizer)
    val_loss = validate(embed_to_score, val_loader, criterion)
    if epoch % 10 == 0:  # Print every 10 epochs
        print(f"Epoch {epoch}: Train Loss = {train_loss:.4f}, Val Loss = {val_loss:.4f}")
    
    # Early stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        epochs_no_improve = 0
        torch.save(embed_to_score.state_dict(), 'best_model.pth')  # Save best model
    else:
        epochs_no_improve += 1
        if epochs_no_improve == patience:
            print("Early stopping!")
            break

100%|██████████| 12/12 [00:00<00:00, 331.46it/s]


Epoch 0: Train Loss = 1.3267, Val Loss = 0.7117


100%|██████████| 12/12 [00:00<00:00, 390.34it/s]


Epoch 10: Train Loss = 1.3379, Val Loss = 0.7100


100%|██████████| 12/12 [00:00<00:00, 402.57it/s]


...

Epoch 360: Train Loss = 1.0825, Val Loss = 0.7013


100%|██████████| 12/12 [00:00<00:00, 369.66it/s]


Epoch 370: Train Loss = 1.0204, Val Loss = 0.7016


100%|██████████| 12/12 [00:00<00:00, 369.21it/s]

Early stopping!


Model took multiple training runs to stop - about 2400 epochs total. Thankfully each epoch is fast and can be done on cpu. 
Patience is there to prevent overfitting - when validation loss increases but training loss goes down, thats overfitting
I realised that if patience is too low, model will stop in a local minima for validation loss. Patience = 50 worked well here. 

Testing best model weights on random internet picture. At least the grade falls below 1 and 5. 

In [72]:
#Testing the trained model
#This tensor is converted from picture of a man https://www.123rf.com/photo_117285262_cheerful-male-person-wearing-colorful-clothes-while-posing-on-camera.html
sample_tensor = torch.FloatTensor([ 1.6438e-02,  2.1896e-02, -8.0362e-02,  2.8341e-02, -2.8072e-02,
         -2.9751e-02,  3.3327e-03,  7.1567e-03, -3.3463e-03, -1.4960e-02,
          1.6220e-02, -5.9948e-02, -3.5069e-03, -3.2648e-02, -4.8549e-02,
          5.3182e-02, -1.2044e-01,  1.3931e-02, -2.7137e-03, -4.5740e-02,
         -5.4338e-03, -7.7437e-03,  1.9806e-03,  3.1177e-02,  2.3815e-02,
         -4.8724e-02,  3.4536e-02,  3.0051e-02,  7.7540e-02,  1.6905e-03,
          8.1902e-02,  2.0170e-02,  3.2379e-02, -5.5702e-04,  6.4838e-03,
          1.6441e-02, -7.1309e-03,  3.3163e-02,  7.0299e-03, -5.3777e-02,
          2.1970e-02, -4.3867e-02,  2.1012e-02,  1.3051e-02, -2.8605e-02,
         -2.1673e-02, -7.7806e-02, -5.0445e-03, -1.7270e-03,  3.8109e-02,
         -1.3078e-02, -9.2852e-04,  1.9087e-02, -1.4945e-02, -6.1000e-02,
          1.8101e-02, -1.5107e-02, -2.0382e-03, -1.2078e-01, -5.7932e-03,
         -8.0106e-02, -1.3348e-02, -1.6996e-02, -4.8043e-02, -4.8187e-02,
         -8.3656e-03,  1.7748e-02, -7.7310e-02,  1.2552e-02, -2.7422e-03,
         -5.8433e-02, -1.9581e-02, -1.2393e-03, -3.2840e-02,  1.9550e-02,
         -6.0912e-02,  2.5954e-02,  2.7044e-02,  1.5751e-02, -2.9800e-02,
         -5.7729e-03,  3.0403e-02, -2.5153e-02,  3.2811e-02,  2.6373e-02,
          3.6481e-02,  1.8692e-02,  1.2436e-02,  1.3068e-02, -5.4420e-02,
          3.5197e-02,  3.0339e-03, -2.9471e-02, -1.6294e-02,  1.9376e-03,
         -9.6888e-03, -9.2772e-04,  2.3004e-02,  1.8474e-02, -1.9492e-02,
         -1.8044e-02, -4.1529e-02, -1.5912e-03, -8.9523e-03,  5.9890e-02,
         -4.0150e-02, -2.6220e-02,  3.3787e-03, -1.3667e-02, -4.1747e-02,
         -6.0041e-02, -8.8984e-03, -4.2063e-03,  5.9290e-03, -2.0958e-02,
         -5.1528e-02, -1.1521e-02,  1.0781e-02,  1.7129e-02,  1.6626e-02,
          2.0090e-02, -1.3816e-02, -1.2629e-02,  3.5362e-02, -5.0381e-03,
         -4.6351e-02, -7.0735e-02, -3.3706e-04,  3.9993e-02, -8.3239e-03,
          6.3205e-04,  3.1786e-02,  5.0934e-02, -4.0317e-02, -9.7049e-02,
          4.1309e-03, -1.8061e-02,  2.9901e-03,  7.8605e-03,  1.7492e-02,
          2.6236e-03,  1.6291e-02, -3.4488e-02,  2.6159e-02, -9.3525e-03,
          4.2920e-02, -1.0373e-02,  1.3415e-02,  2.3318e-02, -4.4368e-02,
         -6.5866e-02,  4.0162e-03,  1.8135e-01,  3.4263e-02, -4.5111e-03,
          2.4679e-02, -2.6330e-02,  3.1580e-03, -4.3842e-02, -7.0146e-03,
          3.0014e-02,  2.1881e-02, -2.8969e-02,  7.2498e-03,  5.8582e-02,
         -4.8642e-02,  6.9074e-04, -1.2843e-01, -5.9926e-02, -1.6402e-02,
         -2.7898e-02, -1.1446e-02, -5.4659e-03,  1.5569e-02, -1.4688e-02,
          8.8983e-02,  2.7876e-02, -1.3252e-02,  8.6685e-02,  2.8667e-02,
          4.4407e-02,  4.5487e-03, -1.2681e-02, -1.2898e-02, -4.0447e-02,
          8.4092e-03,  8.4929e-02,  2.8877e-02, -2.1838e-02, -2.8689e-02,
          8.2374e-04, -4.7018e-02, -4.2617e-02,  4.0515e-02,  2.4294e-02,
          3.6246e-02, -3.2249e-02, -5.6838e-02,  2.5454e-05,  6.8819e-02,
          1.0482e-01, -6.8002e-02,  5.7314e-02,  1.6914e-02, -2.3342e-02,
          1.6111e-02, -1.4589e-02, -4.0899e-02,  1.0747e-02, -2.0445e-02,
          3.9802e-03, -1.4422e-02, -1.3104e-02, -7.0705e-03,  1.0223e-03,
         -1.1471e-02, -6.6757e-03, -3.8847e-02, -3.6280e-02,  5.1143e-02,
          1.7621e-02, -1.5823e-02,  4.1379e-03,  1.6249e-02,  7.7959e-02,
          3.1737e-02,  7.7840e-03, -3.6849e-02,  3.9102e-03, -2.1117e-02,
          1.7891e-02, -1.8777e-02, -2.6012e-02,  4.8590e-02,  1.3336e-01,
         -1.3727e-02, -6.1243e-02, -8.3668e-03,  2.3006e-03,  2.0935e-02,
          2.9661e-02, -7.2977e-02,  3.5479e-02, -2.4728e-02,  1.6162e-03,
         -4.0983e-02, -2.1773e-02, -1.2003e-02, -1.6234e-02,  4.2936e-02,
         -4.7685e-02, -1.6079e-02,  3.3041e-02, -5.8450e-02, -5.6025e-02,
          7.9667e-03, -3.1043e-02,  2.5277e-02,  1.2441e-02, -1.0250e-02,
         -2.4603e-02, -2.7787e-02,  4.4998e-02,  2.5554e-03, -3.1553e-02,
          2.2477e-02, -3.2626e-02,  5.9970e-02,  9.5149e-03, -1.0181e-02,
         -8.6941e-03, -1.7810e-02,  2.7997e-02, -4.9730e-04, -1.5720e-02,
         -1.7308e-03,  1.3384e-02,  3.0308e-02, -2.9071e-02, -6.0074e-03,
         -2.7530e-02, -2.1719e-02,  2.7147e-03,  2.9770e-02,  1.8560e-02,
         -2.8023e-02,  3.4007e-03,  3.8103e-02,  7.9685e-03,  2.2906e-02,
          9.8222e-03,  2.1956e-03,  1.8347e-02, -1.1035e-02, -2.1805e-02,
          8.0348e-03,  6.5365e-03,  5.7519e-03, -9.0100e-03,  3.4582e-03,
         -8.5428e-02, -8.2064e-03,  3.7818e-02, -9.2870e-03,  4.3539e-02,
         -1.0486e-02,  3.6035e-02, -1.8147e-01, -3.0396e-02,  1.1968e-02,
         -6.5025e-02, -8.5649e-02, -3.7049e-02, -2.0510e-02,  4.6033e-02,
          2.8916e-02, -1.9188e-02, -4.0011e-02, -1.7165e-02, -4.7628e-02,
         -5.9055e-03, -4.4336e-02, -4.5208e-02,  3.4894e-02, -3.3236e-03,
          3.5319e-03, -8.0306e-02, -3.3924e-02,  1.5684e-02, -1.4497e-02,
         -1.3824e-02,  1.3490e-02, -1.6188e-02,  2.1202e-02,  4.4270e-03,
          6.1059e-04, -3.2296e-02, -2.4578e-02, -5.8685e-02, -3.1978e-02,
          7.9443e-03, -2.7962e-03,  1.5279e-02, -1.8560e-02,  1.3277e-02,
         -1.8105e-02,  7.7305e-04, -2.1878e-02, -3.5787e-02,  4.1071e-02,
          6.1255e-02,  1.2895e-02, -2.7177e-02, -2.2769e-02, -6.8263e-02,
          8.7712e-03, -2.8485e-02,  7.2261e-03, -3.1435e-02, -4.9915e-02,
          4.9357e-03, -8.0902e-03, -3.0833e-02,  2.5438e-02,  2.3160e-02,
          3.1968e-03, -9.8436e-03,  3.6580e-02, -2.1161e-04, -3.0524e-02,
          3.5791e-02,  3.6604e-02,  1.0698e-02,  2.0288e-02, -1.0971e-02,
         -9.5724e-02,  1.1956e-02, -4.6368e-03,  1.8884e-02, -3.6222e-02,
          2.5449e-02, -3.3758e-02, -2.1567e-02, -4.5111e-03,  3.0986e-02,
          5.4160e-02, -3.9285e-03,  1.7523e-02,  1.0818e-02,  5.6329e-03,
         -2.1182e-03, -7.9104e-02,  4.3279e-02, -5.3102e-03, -4.8174e-02,
         -4.6041e-03, -1.1762e-01, -8.3034e-03,  1.3346e-02, -9.3953e-03,
          5.1457e-02, -1.1119e-02,  7.3139e-03, -2.0534e-02, -1.1732e-02,
          2.6643e-02, -9.7342e-03,  4.2314e-02, -6.7216e-02,  2.8781e-02,
          2.2335e-02, -1.6726e-02,  2.2697e-02,  2.6102e-02,  3.4108e-02,
         -1.3881e-02, -2.6161e-02, -4.4447e-02, -4.9447e-02, -2.6481e-03,
          8.4490e-03, -1.9126e-02, -7.8287e-02, -2.5057e-02,  3.7006e-02,
         -2.3662e-03, -8.1124e-03, -2.2771e-02, -2.3840e-02,  1.6940e-02,
          4.4685e-02, -6.3397e-03,  3.7412e-03,  1.9873e-02,  7.6575e-02,
          1.5466e-02, -2.6530e-02,  7.1482e-03, -2.8864e-03,  1.0379e-02,
         -9.7083e-03,  1.5241e-02,  5.2368e-02, -7.9797e-02,  1.1748e-02,
         -1.8466e-02,  1.5176e-03,  5.3380e-02, -3.0241e-02, -1.9172e-03,
          4.2681e-03,  5.1388e-03,  6.1217e-03,  1.9138e-02,  4.0037e-03,
         -4.4822e-02, -1.5271e-02, -3.2483e-02, -7.6644e-02,  8.8780e-03,
         -1.3251e-02, -6.0519e-03, -2.6461e-02,  7.2956e-03,  4.7201e-02,
          9.8676e-03,  1.4828e-02,  3.8183e-02, -3.2692e-02,  1.6484e-02,
         -3.1506e-02, -8.2864e-02,  1.8854e-02, -2.4680e-02, -5.8000e-02,
          7.4619e-02,  4.6146e-02, -2.2261e-02, -3.5827e-02, -1.4187e-02,
          2.3814e-02,  7.2704e-02, -1.6533e-03, -1.1226e-02, -6.5184e-03,
         -3.2976e-02,  1.2230e-02, -1.3260e-02, -1.3116e-02,  8.0362e-03,
         -1.7711e-02,  6.4595e-02,  2.1063e-02,  1.2096e-02, -5.1930e-02,
          5.4860e-02, -2.2176e-02,  4.1113e-02, -1.5210e-02,  5.1906e-03,
         -2.6861e-02, -2.8147e-02,  3.4399e-02, -4.9842e-02,  2.5388e-02,
          2.6219e-02,  4.1337e-02, -2.2133e-02,  2.3895e-02, -1.2442e-02,
          1.0604e-02,  1.8947e-02]).unsqueeze(0).to(device)
embed_to_score.load_state_dict(torch.load('best_model.pth'))
with torch.no_grad():
    embed_to_score.eval()
    output = embed_to_score(sample_tensor.to(device))
    print(output.item())

3.159768581390381


The embedding (hardcoded here) was generated by running this code in embed.ipynb

image = Image.open(r'C:\Users\rwwj8\OneDrive\Documents\colourful_man.jpg').convert("RGB")
text = "a man wearing green bandana, purple hoodie and long orange trousers"
image_embeds, text_embeds = run_embedding([image], [text])
combined_embeds = combine_embeddings(image_embeds, text_embeds)
print("Image Embedding Shape:", image_embeds.shape)
print(combined_embeds)